# HW 2 - Naive Bayes in Hadoop MR
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the live sessions for week 2 and week 3 you got some practice designing and debugging Hadoop Streaming jobs. In this homework we'll use Hadoop MapReduce to implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your implementation you'll test it on a small dataset that matches the 'Chinese Example' in the _Manning, Raghavan and Shutze_ reading for Week 2. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. By the end of this assignment you should be able to:
* __... describe__ the Naive Bayes algorithm including both training and inference.
* __... perform__ EDA on a corpus using Hadoop MR.
* __... implement__ parallelized Naive Bayes.
* __... constrast__ partial, unordered and total order sort and their implementations in Hadoop Streaming.
* __... explain__ how smoothing affects the bias and variance of a Multinomial Naive Bayes model.

As always, your work will be graded both on the correctness of your output and on the clarity and design of your code. __Please refer to the `README` for homework submission instructions.__ 

## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# global vars (paths) - ADJUST AS NEEDED
JAR_FILE = "/usr/lib/hadoop/hadoop-streaming.jar"
HDFS_DIR = "/user/root/HW2/"
HOME_DIR = "/w261_personal/Assignments/HW2/" # FILL IN HERE eg. /media/notebooks/Assignments/HW2

In [3]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [4]:
# data path
ENRON = "data/enronemail_1h.txt"

In [5]:
!hdfs dfs -mkdir "/user/root"

mkdir: `/user/root': File exists


In [6]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -ls 
# !hdfs dfs -mkdir {HDFS_DIR}

Found 1 items
drwxr-xr-x   - root hadoop          0 2022-05-30 17:53 HW2


In [7]:
# !mkdir {HOME_DIR}/data

In [8]:
!gsutil cp gs://w261-hw-data/main/Assignments/HW2/data/enronemail_1h.txt {HOME_DIR}data/enronemail_1h.txt

Copying gs://w261-hw-data/main/Assignments/HW2/data/enronemail_1h.txt...
/ [1 files][199.8 KiB/199.8 KiB]                                                
Operation completed over 1 objects/199.8 KiB.                                    


# Question 1: Hadoop MapReduce Key Takeaways.  

This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. This question will briefly highlight some of the most important concepts that you need to understand about Hadoop Streaming and MapReduce before we move on to Spark next week.   

### Q1 Tasks:

* __a) short response:__ What "programming paradigm" is Hadoop MapReduce based on? What are the main ideas of this programming paradigm and how does MapReduce exemplify these ideas?

* __b) short response:__ What is the Hadoop Shuffle? When does it happen? Why is it potentially costly? Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.

* __c) short response:__ In Hadoop Streaming why do the input and output record format of a combiner script have to be the same? [__`HINT`__ _what level of combining does the framework guarantee? what is the relationship between the record format your mapper emits and the format your reducer expects to receive?_]

* __d) short response:__ To what extent can you control the level of parallelization of your Hadoop Streaming jobs? Please be specific.

* __e) short response:__ What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop? 

### Q1 Student Answers:

> __a)__ Hadoop Mapreduce is based off of the the functional programming paradigm. The main ideas of functional programming is that each tasks are executed sequentially. MapReduce exemplifies this since there are several stages within a MapReduce job, each of them being executed after another stage is complete.

> __b)__ The shuffle is the step between the map phase and the reduce phase. It takes the outputs from the map stage, and sorts the results so they can be reduced. It is potentially costly because it has to iterate through every observation within your dataset and place it a pre-defined order. Iterating through every observation is a potentially costly computation. One step we can take to reduce this cost is to pre-shuffle and combine our observations within each partition of the map phase, also called a combiner. This reduces the cost since you are reducing the number of records that you iterate over.

> __c)__ Input and output records of a combiner script have to be the same because a reducer expects to receive as an input the output of the map stage. Because of this, a combiner has to act like it is an output a mapper, and since the input of a combiner is a mapper, it needs to produce an output of the same format.

> __d)__ 1. You can control the number of partitions that are parallelized within a Hadoop Streaming job, but you can't control whether each of the parallelized tasks are completed at the same pace. Some partitions may be more computational costly compared to other partitions, due to differences in the data within the partitions. On top of this, some partitions may be computed on faster, more reliable machines, resulting in uneven completion times. Because of these issues, you need to be careful about the sequencing of jobs within a Hadoop Stream. You can mitigate these factors by adding in parameters that wait until all partitions are completed before moving on to the next phase.

> __e)__ Networked computing clusters prompted the creation of parallel computation frameworks, since having the capability to scale computations across a network of computers could speed up calculations linearally to the number of computers in the cluster.

# Question 2: MapReduce Design Patterns.  

In the last two live sessions and in your readings from Lin & Dyer you encountered a number of techniques for manipulating the logistics of a MapReduce implementation to ensure that the right information is available at the right time and location. In this question we'll review a few of the key techniques you learned.   

### Q2 Tasks:

* __a) short response:__ What are counters (in the context of Hadoop Streaming)? How are they useful? What kinds of counters does Hadoop provide for you? How do you create your own custom counter?

* __b) short response:__ What are composite keys? How are they useful? How are they related to the idea of custom partitioning?

* __c) short response:__ What is the order inversion pattern? What problem does it help solve? How do we implement it? 

### Q2 Student Answers:

> __a)__   

> __b)__ Type your answer here!  

> __c)__ Type your answer here!  


# Question 3: Understanding Total Order Sort

The key challenge in distributed computing is to break a problem into a set of sub-problems that can be performed without communicating with each other. Ideally, we should be able to define an arbirtary number of splits and still get the right result, but that is not always possible. Parallelization becomes particularly challenging when we need to make comparisons between records, for example when sorting. Total Order Sort allows us to order large datasets in a way that enables efficient retrieval of results. Before beginning this assignment, make sure you have read and understand the [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb). You can skip the first two MRJob sections, but the rest of section III and all of section IV are **very** important (and apply to Hadoop Streaming) so make sure to read them closely. Feel free to read the Spark sections as well but you won't be responsible for that material until later in the course. To verify your understanding, answer the following questions.

### Q3 Tasks:

* __a) short response:__ What is the difference between a partial sort, an unordered total sort, and a total order sort? From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

* __b) short response:__ Which phase of a MapReduce job is leveraged to implement Total Order Sort? Which default behaviors must be changed. Why must they be changed?

* __c) short response:__ Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.  

* __d) short response:__ Explain why we need to use an inverse hash code function.

* __e) short response:__ Where does this function need to be located so that a Total Order Sort can be performed?

### Q3 Student Answers:

> __a)__ Type your answer here!  

> __b)__ Type your answer here!  

> __c)__ Type your answer here!  

> __d)__ Type your answer here!  

> __e)__ Type your answer here!  

# About the Data
For the main task in this portion of the homework you will train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/enronemail_1h.txt`__).

__Source:__   
The original data included about 93,000 emails which were made public after the company's collapse. There have been a number raw and preprocessed versions of this corpus (including those available [here](http://www.aueb.gr/users/ion/data/enron-spam/index.html) and [here](http://www.aueb.gr/users/ion/publications.html)). The subset we will use is limited to emails from 6 Enron employees and a number of spam sources. It is part of [this data set](http://www.aueb.gr/users/ion/data/enron-spam/) which was created by researchers working on personlized Bayesian spam filters. Their original publication is [available here](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). __`IMPORTANT!`__ _For this homework please limit your analysis to the 100 email subset which we provide. No need to download or run your analysis on any of the original datasets, those links are merely provided as context._

__Preprocessing:__  
For their work, Metsis et al. (the authors) appeared to have pre-processed the data, not only collapsing all text to lower-case, but additionally separating "words" by spaces, where "words" unfortunately include punctuation. As a concrete example, the sentence:  
>  `Hey Jon, I hope you don't get lost out there this weekend!`  

... would have been reduced by Metsis et al. to the form:  
> `hey jon , i hope you don ' t get lost out there this weekend !` 

... so we have reverted the data back toward its original state, removing spaces so that our sample sentence would now look like:
> `hey jon, i hope you don't get lost out there this weekend!`  

Thus we have at least preserved contractions and other higher-order lexical forms. However, one must be aware that this reversion is not complete, and that some object (specifically web sites) will be ill-formatted, and that all text is still lower-cased.


__Format:__   
All messages are collated to a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

where:  
>    `ID = string; unique message identifier`  
    `SPAM = binary; with 1 indicating a spam message`  
    `SUBJECT = string; title of the message`  
    `CONTENT = string; content of the message`   
    
Note that either of `SUBJECT` or `CONTENT` may be "NA", and that all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [9]:
!pwd

/media/notebooks/student-workspace


In [10]:
!head -n 5 {HOME_DIR}{ENRON} | cut -c-100

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [11]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
# !head -n 5 /media/notebooks/Assignments/HW2/{ENRON} | cut -c-100

In [12]:
!wc -l {HOME_DIR}{ENRON}

100 /w261_personal/Assignments/HW2/data/enronemail_1h.txt


In [13]:
# see how many messages/lines are in the file 
#(this number may be off by 1 if the last line doesn't end with a newline)
# !wc -l /media/notebooks/Assignments/HW2/{ENRON}

In [14]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir {HDFS_DIR}

mkdir: `/user/root/HW2': File exists


In [15]:
!hdfs dfs -copyFromLocal {HOME_DIR}{ENRON} {HDFS_DIR}/enron.txt

copyFromLocal: `/user/root/HW2/enron.txt': File exists


In [16]:
# load the data into HDFS (RUN THIS CELL AS IS)
# !hdfs dfs -copyFromLocal /media/notebooks/Assignments/HW2/{ENRON} {HDFS_DIR}/enron.txt

In [17]:
!hdfs dfs -ls {HDFS_DIR}

Found 4 items
drwxr-xr-x   - root hadoop          0 2022-05-30 17:53 /user/root/HW2/chinese-output
-rw-r--r--   1 root hadoop        119 2022-05-30 16:43 /user/root/HW2/chineseTest.txt
-rw-r--r--   1 root hadoop        107 2022-05-30 16:43 /user/root/HW2/chineseTrain.txt
-rw-r--r--   1 root hadoop     204559 2022-05-30 16:39 /user/root/HW2/enron.txt


# Question 4:  Enron Ham/Spam EDA.
Before building our classifier, lets get aquainted with our data. In particular, we're interested in which words occur more in spam emails than in legitimate ("ham") emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. You'll also learn about two new Hadoop streaming parameters that will allow you to control how the records output from your mappers are partitioned for reducing on separate nodes. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Q4 Tasks:
* __a) code:__ Complete the missing components of the code in __`EnronEDA/mapper.py`__ and __`EnronEDA/reducer.py`__ to create a Hadoop  MapReduce job that counts how many times each word in the corpus occurs in an email for each class. Pay close attention to the data format specified in the docstrings of these scripts _-- there are a number of ways to accomplish this task, we've chosen this format to help illustrate a technique in `part e`_. Run the provided unit tests to confirm that your code works as expected, then run the provided Hadoop Streaming command to apply your analysis to the Enron data.


* __b) code + short response:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`a`' and then report the answer in the space provided.)


* __c) short response:__ Would it have been possible to add some sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count? Explain why or why not. (`HINT:` This question demands an understanding of the sequence of the phases of MapReduce.)


* __d) code + short response:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file. Then describe in words how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reodering the records from `part a` we don't need to write a mapper or reducer, just use `/bin/cat` for both_)


* __e) code:__ A more efficient alternative to '`grep`-ing' for the top 10 words in each class would be to use the Hadoop framework to separate records from each class into its own partition so that we can just read the top lines in each. Rewrite your job from ` part d` to specify 2 reduce tasks and to tell Hadoop to partition based on the second field (which indicates spam/ham in our data). Your code should maintain the secondary sort -- that is each partition should list words from most to least frequent.

### Q4 Student Answers:
> __b)__ The word "assistance" occurs 2 times in non-spam emails, and 8 times in spam emails. The word is much more likely to occur within spam emails 

> __c)__ Yes, we can add a command that sorts results by count, but the operation can only be conducted after the reduce phase, since any sorting before then would be premature.  

> __d)__ You could print an output of the top 10 words in each class by sorting first by count and then class, while running an ascending and descending sort on class. Then use the head function to print the top ten results. 

In [18]:
# replacement for cell below
!chmod a+x {HOME_DIR}EnronEDA/mapper.py
!chmod a+x {HOME_DIR}EnronEDA/reducer.py

In [19]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
# !chmod a+x EnronEDA/mapper.py
# !chmod a+x EnronEDA/reducer.py

In [17]:
# replacement for cell below
!echo -e "d1	1	title	body\nd2	0	title	body" | {HOME_DIR}EnronEDA/mapper.py 

title	0	1
body	0	1
title	1	1
body	1	1


In [18]:
# replacement for cell below
!echo -e "d1	1	title	body\nd2	0	title	body" | {HOME_DIR}EnronEDA/mapper.py | {HOME_DIR}EnronEDA/reducer.py

title	0	1
body	0	1
title	1	1
body	1	1


In [19]:
# # part a - unit test EnronEDA/mapper.py (RUN THIS CELL AS IS)
# !echo -e "d1	1	title	body\nd2	0	title	body" | EnronEDA/mapper.py

need to pick up the last word
print out the first word correctly
the first row is being excluded

In [20]:
# replacement for cell below
!echo "one	1	1\none	0	1\none	0	1\ntwo	0	1" | {HOME_DIR}EnronEDA/reducer.py

one	0	2
two	0	1
one	1	1


In [24]:
# # part a - unit test EnronEDA/reducer.py (RUN THIS CELL AS IS)
# !echo -e "one	1	1\none	0	1\none	0	1\ntwo	0	1" | EnronEDA/reducer.py

In [25]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [56]:
!cd /w261_personal/Assignments/HW2/

In [57]:
!{HDFS_DIR}

/bin/sh: 1: /user/root/HW2/: not found


In [26]:
# replacement for cell below
!hadoop jar {JAR_FILE} \
  -files {HOME_DIR}EnronEDA/reducer.py,{HOME_DIR}EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop/hadoop-streaming-3.2.2.jar] /tmp/streamjob3958241406881344660.jar tmpDir=null
2022-05-29 21:15:54,645 INFO client.RMProxy: Connecting to ResourceManager at w261-m/10.138.0.22:8032
2022-05-29 21:15:54,882 INFO client.AHSProxy: Connecting to Application History server at w261-m/10.138.0.22:10200
2022-05-29 21:15:55,415 INFO client.RMProxy: Connecting to ResourceManager at w261-m/10.138.0.22:8032
2022-05-29 21:15:55,415 INFO client.AHSProxy: Connecting to Application History server at w261-m/10.138.0.22:10200
2022-05-29 21:15:55,646 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1653840139532_0005
2022-05-29 21:15:56,021 INFO mapred.FileInputFormat: Total input files to process : 1
2022-05-29 21:15:56,485 INFO mapreduce.JobSubmitter: number of splits:9
2022-05-29 21:15:56,682 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1653840139532_0005
2022-05-29 21:15:56,684 INFO mapred

In [43]:
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
# !hadoop jar {JAR_FILE} \
#   -files EnronEDA/reducer.py,EnronEDA/mapper.py \
#   -mapper mapper.py \
#   -reducer reducer.py \
#   -input {HDFS_DIR}/enron.txt \
#   -output {HDFS_DIR}/eda-output \
#   -numReduceTasks 2 \
#   -cmdenv PATH={PATH}

Exception in thread "main" java.io.FileNotFoundException: File EnronEDA/reducer.py does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:668)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:989)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:658)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:458)
	at org.apache.hadoop.util.GenericOptionsParser.validateFiles(GenericOptionsParser.java:454)
	at org.apache.hadoop.util.GenericOptionsParser.validateFiles(GenericOptionsParser.java:399)
	at org.apache.hadoop.util.GenericOptionsParser.processGeneralOptions(GenericOptionsParser.java:331)
	at org.apache.hadoop.util.GenericOptionsParser.parseGeneralOptions(GenericOptionsParser.java:572)
	at org.apache.hadoop.util.GenericOptionsParser.<init>(GenericOptionsParser.java:174)
	at org.apache.hadoop.util.GenericOptionsParser.<init>(GenericOp

In [59]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > {HOME_DIR}EnronEDA/results.txt

^C


In [60]:
# part b - write your grep command here
!grep assistance {HOME_DIR}EnronEDA/results.txt

In [ ]:
# part d - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

In [28]:
# part d - write your Hadoop streaming job here
!sort -k2,2n {HOME_DIR}EnronEDA/results.txt > {HOME_DIR}EnronEDA/results_class_sorted.txt
!sort -k3,3rn {HOME_DIR}EnronEDA/results.txt > {HOME_DIR}EnronEDA/results_count_sorted.txt

In [28]:
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=“-k2,2nr -k3,3nr” \
  -files {HOME_DIR}EnronEDA/reducer.py,{HOME_DIR}EnronEDA/mapper.py \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH} 

2022-05-29 16:20:47,014 ERROR streaming.StreamJob: Unrecognized option: -k3,3nr”
Usage: $HADOOP_HOME/bin/hadoop jar hadoop-streaming.jar [options]
Options:
  -input          <path> DFS input file(s) for the Map step.
  -output         <path> DFS output directory for the Reduce step.
  -mapper         <cmd|JavaClassName> Optional. Command to be run as mapper.
  -combiner       <cmd|JavaClassName> Optional. Command to be run as combiner.
  -reducer        <cmd|JavaClassName> Optional. Command to be run as reducer.
  -file           <file> Optional. File/dir to be shipped in the Job jar file.
                  Deprecated. Use generic option "-files" instead.
  -inputformat    <TextInputFormat(default)|SequenceFileAsTextInputFormat|JavaClassName>
                  Optional. The input format class.
  -outputformat   <TextOutputFormat(default)|JavaClassName>
                  Optional. The output format class.
  -partitioner    <JavaClassName>  Optional. The partitioner class.
  -numReduceTa

In [29]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

rm: `/user/root/HW2//eda-sort-output': No such file or directory


In [ ]:
# part e - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -files {HOME_DIR}EnronEDA/reducer.py,{HOME_DIR}EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

In [ ]:
# part e - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head

__Expected output:__
<table>
<th>part-00000:</th>
<th>part-00001:</th>
<tr><td><pre>
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135
</pre></td>
<td><pre>
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153
</pre></td></tr>
</table>

# Question 5: Counters and Combiners.
Tuning the number of mappers & reducers is helpful to optimize very large distributed computations. Doing so successfully requires a thorough understanding of the data size at each stage of the job. As you learned in the week3 live session, counters are an invaluable resource for understanding this kind of detail. In this question, we will take the EDA performed in Question 4 as an opportunity to illustrate some related concepts.

### Q5 Tasks:
* __a) short response:__ Read the Hadoop output from your job in Question 4a to report how many records are emitted by the mappers and how many records are received be the reducers. In the context of word counting what does this number represent practically?

* __b) code:__ Note that we wrote the reducer in question 4a such that the input and output record format is identical. This makes it easy to use the same reducer script as a combiner. In the space provided below, write the Hadoop Streaming command to re-run your job from question 4a with this combining added.

* __c) short response__: Report the number of records emitted by your mappers in part b and the number of records received by your reducers. Compare your results here to what you saw in part a. Explain.

* __d) short response__: Describe a scenario where using a combiner would _NOT_ improve the efficiency of the shuffle stage. Explain. [__`BONUS:`__ how does increasing the number of mappers affect the usefulness of a combiner?]

### Q5 Student Answers:
> __a)__ The mapper output and reducer input have the same number of records: 11,433. In terms of word counting, 

> __c)__ There are 11,433 records output by the mapper, but only 9,078 records recieved by the reducer. Since there are ~2,000 fewer records to process by the reducer compared to not using a combiner, there is a large effeciency gain here.  

> __d)__ One scenario where using a combiner might not improve effeciency is if there are not many records to combine within each partition. This would not be effecient since you would spend computing resources iterating over every record, but you would not have any improved results from it. This may happen if you have many partitions with few records in each partition.

In [30]:
# part b - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [31]:
# part b - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -files {HOME_DIR}EnronEDA/reducer.py,{HOME_DIR}EnronEDA/mapper.py \
  -mapper mapper.py \
  -combiner reducer.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop/hadoop-streaming-3.2.2.jar] /tmp/streamjob3193006591635962875.jar tmpDir=null
2022-05-29 17:28:49,877 INFO client.RMProxy: Connecting to ResourceManager at w261-m/10.138.0.22:8032
2022-05-29 17:28:50,092 INFO client.AHSProxy: Connecting to Application History server at w261-m/10.138.0.22:10200
2022-05-29 17:28:50,572 INFO client.RMProxy: Connecting to ResourceManager at w261-m/10.138.0.22:8032
2022-05-29 17:28:50,573 INFO client.AHSProxy: Connecting to Application History server at w261-m/10.138.0.22:10200
2022-05-29 17:28:50,765 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1653840139532_0003
2022-05-29 17:28:51,573 INFO mapred.FileInputFormat: Total input files to process : 1
2022-05-29 17:28:51,625 INFO mapreduce.JobSubmitter: number of splits:9
2022-05-29 17:28:51,816 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1653840139532_0003
2022-05-29 17:28:51,817 INFO mapred

# Question 6: Document Classification Task Overview.
The week 2 assigned reading from Chapter 13 of _Introduction to Information Retrieval_ by Manning, Raghavan and Schutze provides a thorough introduction to the document classification task and the math behind Naive Bayes. In this question we'll use the example from Table 13.1 (reproduced below) to 'train' an unsmoothed Multinomial Naive Bayes model and classify a test document by hand.

<table>
<th>DocID</th>
<th>Class</th>
<th>Subject</th>
<th>Body</th>
<tr><td>Doc1</td><td>1</td><td></td><td>Chinese Beijing Chinese</td></tr>
<tr><td>Doc2</td><td>1</td><td></td><td>Chinese Chinese Shanghai</td></tr>
<tr><td>Doc3</td><td>1</td><td></td><td>Chinese Macao</td></tr>
<tr><td>Doc4</td><td>0</td><td></td><td>Tokyo Japan Chinese</td></tr>
</table>

### Q6 Tasks:
* __a) short response:__ Equation 13.3 in Manning, Raghavan and Shutze shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$? Your answer should include both mathematical notatation and verbal explanation.


* __b) short response:__ The Enron data includes two classes of documents: `spam` and `ham` (they're actually labeled `1` and `0`). In plain English, explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data. How will we would estimate these values from a training corpus? How many passes over the data would we need to make to retrieve this information for all classes and all words?


* __c) hand calculations:__ Above we've reproduced the document classification example from the textbook (we added an empty subject field to mimic the Enron data format). Remember that the classes in this "Chinese Example" are `1` (about China) and `0` (not about China). Calculate the class priors and the conditional probabilities for an __unsmoothed__ Multinomial Naive Bayes model trained on this data. Show the calculations that lead to your result using markdown and $\LaTeX$ in the space provided or by embedding an image of your hand written work. [`NOTE:` _Your results should NOT match those in the text -- they are training a model with +1 smoothing you are training a model without smoothing_]


* __d) hand calculations:__ Use the model you trained to classify the following test document: `Chinese Chinese Chinese Tokyo Japan`. Show the calculations that lead to your result using markdown and   $\LaTeX$ in the space provided or by embedding an image of your hand written work.


* __e) short response:__ Compare the classification you get from this unsmoothed model in `d`/`e` to the results in the textbook's "Example 1" which reflects a model with Laplace plus 1 smoothing. How does smoothing affect our inference?

### Q6 Student Answers:
> __a)__ The two pieces of information required to calculated $\hat{P}(c|d)$ is word count of each word within a document, and the probabilty of each of those words to appear in a document of each class. In mathematical terms, the probability of each word to appear within each class is $\hat{P}(t_{k} | c)$. Using those two pieces of information, you sum the probabilities for each word for each class, and choose the class with greater probability. In mathematical terms this means, $argmax\ sum \hat{P}(t_{k} | c)$

> __b)__ $\hat{P}(t_{k} | c)$ means the probability of a word occurring in spam email and a ham email. In other words, you count the number of times that word appears in both a spam and ham email, and divide that by the total number of spam and ham emails. $\hat{P}(c)$ is just the sum of the $\hat{P}(t_{k} | c)$ probabilities for each word within an email. Each email recieves a $\hat{P}(c)$ value for each class.

> __c)__ Show your calculations here using markdown & $\LaTeX$ or embed them below!

> __d)__ Show your calculations here using markdown & $\LaTeX$ or embed them below!

> __e)__ Laplace smoothing results in higher probabilites for both classes, but ultimately returns the same results compared to when you don't include Laplace smoothing. 


In [ ]:
# part d/e - if you didn't write out your calcuations above, embed a picture of them here:
from IPython.display import Image
Image(filename="path-to-hand-calulations-image.png")

# Question 7: Naive Bayes Inference.
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the Chinese example corpus from the textbook to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Q7 Tasks:
* __a) short response:__ run the provided cells to create the example files and load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the 'keys' and 'values' in this file? Which record means something slightly different than the rest? The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they? [`HINT`: _This file represents the model from Example 13.1 in the textbook, if you're having trouble getting oriented try comparing our file to the numbers in that example._]


* __b) short response:__ When performing Naive Bayes in practice instead of multiplying the probabilities (as in equation 13.3) we add their logs (as in equation 13.4). Why do we choose to work with log probabilities? If we had an unsmoothed model, what potential error could arise from this transformation?


* __c) short response:__ Documents 6 and 8 in the test set include a word that did not appear in the training corpus (and as a result does not appear in the model). What should we do at inference time when we need a class conditional probability for this word?


* __d) short response:__ The goal of our MapReduce job is to stream over the test set and classify each document by peforming the calculation from equation 13.4. To do this we'll load the model file (which contains the probabilities for equation 13.4) into memory on the nodes where we do our mapping. This is called an in-memory join. Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not. From a scability perspective when would this kind of memory use be justified? when would it be unwise?


* __e) code:__ Complete the code in __`NaiveBayes/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected then write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:` _Don't forget to add the model file to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]


* __f) short response:__ In our test example and in the Enron data set we have fairly short documents. Since these fit fine in memory on a mapper node we didn't need a reducer and could just do all of our calculations in the mapper. However with much longer documents (eg. books) we might want a higher level of parallelization -- for example we might want to process parts of a document on different nodes. In this hypothetical scenario how would our algorithm design change? What could the mappers still do? What key-value structure would they emit? What would the reducers have to do as a last step?

### Q7 Student Answers:
> __a)__ Within NBmodel.txt, the keys are each of the words that appear within the training set, and there are several sets of values. In order of appearce, they are count of times of appearence within each of the classes (spam and not spam), and then the conditional probability for that word for each class. 

> The record that is slightly different is the ClassPriors records, which acts somewhat as a total row. The values within this record are; number of records for each class, and then the class prior of each class.

> The two fields within each record that we don't necessarily need for modeling are the counts of each word within each class. These are used to calculate the conditional probabilities, and will be useful for debugging since conditional probabilities are tougher to interpret.

> __b)__ We choose to work with adding log probabilities instead of multiplying, since the multiplying probabilities may results in numbers that are too small to store in memory, also known as floating point underflow, results in incorrect calculations. One downside for this transformation is that if you don't smooth your model, you calculation may result in undefined if there is probability of zero, since the log of zero is undefined.

> __c)__ At inference time, these two words should be ignored, since they do not have a probability associated with either class, and there is no harm caused by dropping them from the model.

> __d)__ Loading a model state into memory is a departure from functional programming, since you are moving beyond a string of functions, and loading a function that has been pre-calculated/trained. This approach most likely would not scale well if the corpus gets large enough. Say you have a corpus the size of the english language, and you join that against each partition, you are most likely going to get an "out of memory" error. Clever ways of partioning your corpus/joins most likely will allow you to circumvent this issue. 

> __e)__ Complete the coding portion of this question before answering 'f'.

> __f)__ If we switch to an algorithm with higher levels of parallelization, one of the main changes we'd have to make is it switch the class prediction to the reducer, after results have been aggregated. This is because you wouldn't have the full corpus to decide whether the probability of one class is higher than the other. The mappers could still begin aggregating log probabilities for words within the document, you just couldn't choose a predicted class until the reducer phase. For the reducer to work, the key-value output of the mapper would have to be docID, class, and then aggregated log probabilites for both classes. The reducer would then combine the log probabilites for each docID-class combination.

Run these cells to create the example corpus and model.

In [18]:
%%writefile {HOME_DIR}NaiveBayes/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Overwriting /w261_personal/Assignments/HW2/NaiveBayes/chineseTrain.txt


In [19]:
%%writefile {HOME_DIR}NaiveBayes/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Overwriting /w261_personal/Assignments/HW2/NaiveBayes/chineseTest.txt


In [20]:
%%writefile {HOME_DIR}NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Overwriting /w261_personal/Assignments/HW2/NBmodel.txt


In [21]:
# load the data files into HDFS
!hdfs dfs -copyFromLocal {HOME_DIR}NaiveBayes/chineseTrain.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal {HOME_DIR}NaiveBayes/chineseTest.txt {HDFS_DIR}

Your work for `part e` starts here:

In [22]:
# part e - do your work in NaiveBayes/classify_mapper.py first, then run this cell.
!chmod a+x {HOME_DIR}NaiveBayes/classify_mapper.py

In [34]:
!cat {HOME_DIR}NaiveBayes/classify_mapper.py

#!/usr/bin/env python
"""
Mapper for Naive Bayes Inference.
INPUT:
    ID \t true_class \t subject \t body \n
OUTPUT:
    ID \t true_class \t logP(ham|doc) \t logP(spam|doc) \t predicted_class
SUPPLEMENTAL FILE: 
    This script requires a trained Naive Bayes model stored 
    as NBmodel.txt in the current directory. The model should 
    be a tab separated file whose records look like:
        WORD \t ham_count,spam_count,P(word|ham),P(word|spam)
        
Instructions:
    We have loaded the supplemental file and taken the log of 
    each conditional probability in the model. We also provide
    the code to tokenize the input lines for you. Keep in mind 
    that each 'line' of this file represents a unique document 
    that we wish to classify. Fill in the missing code to get
    the probability of each class given the words in the document.
    Remember that you will need to handle the case where you
    encounter a word that is not represented in the model.
"""
import os
import r

In [54]:
# part e - unit test NaiveBayes/classify_mapper.py (RUN THIS CELL AS IS)
!cat {HOME_DIR}NaiveBayes/chineseTest.txt | {HOME_DIR}NaiveBayes/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [41]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output

rm: `/user/root/HW2//chinese-output': No such file or directory


In [43]:
# part e - write your Hadooop streaming job here
!hadoop jar {JAR_FILE} \
  -files {HOME_DIR}NBmodel.txt,{HOME_DIR}NaiveBayes/classify_mapper.py \
  -mapper classify_mapper.py \
  -numReduceTasks 1 \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop/hadoop-streaming-3.2.2.jar] /tmp/streamjob1398313069743419072.jar tmpDir=null
2022-05-30 17:53:05,741 INFO client.RMProxy: Connecting to ResourceManager at w261-m/10.138.0.23:8032
2022-05-30 17:53:05,970 INFO client.AHSProxy: Connecting to Application History server at w261-m/10.138.0.23:10200
2022-05-30 17:53:06,486 INFO client.RMProxy: Connecting to ResourceManager at w261-m/10.138.0.23:8032
2022-05-30 17:53:06,486 INFO client.AHSProxy: Connecting to Application History server at w261-m/10.138.0.23:10200
2022-05-30 17:53:06,713 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1653928351455_0001
2022-05-30 17:53:07,133 INFO mapred.FileInputFormat: Total input files to process : 1
2022-05-30 17:53:07,276 INFO mapreduce.JobSubmitter: number of splits:10
2022-05-30 17:53:07,506 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1653928351455_0001
2022-05-30 17:53:07,508 INFO mapre

In [44]:
# part e - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > {HOME_DIR}NaiveBayes/chineseResults.txt

In [46]:
# part e - take a look (RUN THIS CELL AS IS)
!cat {HOME_DIR}NaiveBayes/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set:</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 8: Naive Bayes Training.
In Question 7 we used a model that we had trained by hand. Next we'll develop the code to do that same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process take a look at  `6a` and `6b` where you described the pieces of information you'll need to collect in order to encode a Multinomial Naive Bayes model. We now want to retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totalled up _before_ you start estimating the class conditional probabilities for each word. It would be best (i.e. most scalable) if we could find a way to do this tallying without storing the whole vocabulary in memory... Use an appropriate MapReduce design pattern to implement this efficiently! 


### Q8 Tasks:
* __a) make a plan:__  Fill in the docstrings for __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`enronemail_1h.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement it:__ Complete the code in __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own devising). When you are satisfied with your Python code design and run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 6.


* __c) short response:__ We saw in Question 6 that adding Laplace smoothing (where the smoothing parameter $k=1$) makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` see equation 13.7 in Manning, Raghavan and Schutze].


* __d) short response:__ There are a couple of approaches that we could take to handle the extra piece of information you identified in `c`: 1) if we knew this extra information beforehand, we could provide it to our reducer as a configurable parameter for the vocab size dynamically (_where would we get it in the first place?_). Or 2) we could compute it in the reducer without storing any bulky information in memory but then we'd need some postprocessing or a second MapReduce job to complete the calculation (_why?_). Breifly explain what is non-ideal about each of these options. 


* __e) code + short response:__ Choose one of the 2 options above. State your choice & reasoning in the space below then use that strategy to complete the code in __`NaiveBayes/train_reducer_smooth.py`__. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above (and the calculations in the textbook example). __IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. 

    - [`HINT:` You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)] 

    - [`HINT:` Don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications]. 



__IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. [`HINT:`_You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)]


### Q8 Student Answers:
> __ c)__ Type your answer here!

> __ d)__ Type your answer here!

> __ e)__ Type your answer here!


In [19]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x {HOME_DIR}NaiveBayes/train_mapper.py
!chmod a+x {HOME_DIR}NaiveBayes/train_reducer.py
!echo "=========== MAPPER DOCSTRING ============"
!head -n 8 {HOME_DIR}NaiveBayes/train_mapper.py | tail -n 6
!echo "=========== REDUCER DOCSTRING ============"
!head -n 8 {HOME_DIR}NaiveBayes/train_reducer.py | tail -n 6

=========== MAPPER DOCSTRING ============
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    partitionKey \t word \t class0_partialCount,class1_partialCount       
    
=========== REDUCER DOCSTRING ============
Reducer aggregates word counts by class and emits frequencies.

INPUT:
    partitionKey \t word \t class0_partialCount,class1_partialCount 
OUTPUT:
    word \t class0_totalCount,class1_totalCount,class0_condprob,class1_condprob


In [62]:
!cat {HOME_DIR}NaiveBayes/train_mapper.py

#!/usr/bin/env python
"""
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    partitionKey \t word \t class0_partialCount,class1_partialCount       
    

Instructions:
    You know what this script should do, go for it!
    (As a favor to the graders, please comment your code clearly!)
    
    A few reminders:
    1) To make sure your results match ours please be sure
       to use the same tokenizing that we have provided in
       all the other jobs:
         words = re.findall(r'[a-z]+', text-to-tokenize.lower())
         
    2) Don't forget to handle the various "totals" that you need
       for your conditional probabilities and class priors.
       
Partitioning:
    In order to send the totals to each reducer, we need to implement
    a custom partitioning strategy.
    
    We will gene

In [63]:
!cat {HOME_DIR}NaiveBayes/train_reducer.py

#!/usr/bin/env python
"""
Reducer aggregates word counts by class and emits frequencies.

INPUT:
    partitionKey \t word \t class0_partialCount,class1_partialCount 
OUTPUT:
    word \t class0_totalCount,class1_totalCount,class0_condprob,class1_condprob
    
Instructions:
    Again, you are free to design a solution however you see 
    fit as long as your final model meets our required format
    for the inference job we designed in Question 8. Please
    comment your code clearly and concisely.
    
    A few reminders: 
    1) Don't forget to emit Class Priors (with the right key).
    2) In python2: 3/4 = 0 and 3/float(4) = 0.75
"""
##################### YOUR CODE HERE ####################




































##################### (END) CODE HERE ####################

In [64]:
!cat {HOME_DIR}NaiveBayes/train_reducer_smooth.py

#!/usr/bin/env python

import os
import sys                                                  
import numpy as np  

#################### YOUR CODE HERE ###################

































#################### (END) YOUR CODE ###################

__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [56]:
# part b - write a unit test for your mapper here
!cat {HOME_DIR}NaiveBayes/chineseTrain.txt | {HOME_DIR}NaiveBayes/train_mapper.py | sort -k1,1 -k2,2 -k3,3 | column -t #> {HOME_DIR}NaiveBayes/reducer_input_test.txt

A  !class_word_counts  3  8
A  !distinct_words     6  0
A  !doc_counts_class   1  3
A  beijing             0  1
A  chinese             0  1
A  chinese             0  1
A  chinese             0  1
A  chinese             0  1
A  chinese             0  1
A  chinese             1  0
A  japan               1  0
A  macao               0  1
A  shanghai            0  1
A  tokyo               1  0


In [50]:
!cat {HOME_DIR}NaiveBayes/chineseTrain.txt

D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese


In [20]:
!grep -v 'ClassPriors' {HOME_DIR}NaiveBayes/chineseTrain.txt | wc -l

4


In [58]:
# part b - write a unit test for your reducer here

!cat {HOME_DIR}NaiveBayes/reducer_input_test.txt | {HOME_DIR}NaiveBayes/train_reducer.py | column -t

beijing      0,1.0,0,0,0,0.3333333333333333
chinese      1.0,5.0,0,0,0.3333333333333333,1.0
japan        1.0,0,0,0,0.3333333333333333,0
macao        0,1.0,0,0,0,0.3333333333333333
shanghai     0,1.0,0,0,0,0.3333333333333333
tokyo        1.0,0,0,0,0.3333333333333333,0
ClassPriors


In [ ]:
# part b - write a systems test for your mapper + reducer together here

In [ ]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model

In [ ]:
# part b - write your hadoop streaming job

In [ ]:
# part b - extract your results (i.e. model) to a local file

In [ ]:
# part b - print your model so that we can confirm that it matches expected results

In [ ]:
!cat {HOME_DIR}NaiveBayes/chineseTrain.txt

__`part e starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [ ]:
# part e - write a unit test for your NEW reducer here

In [ ]:
# part e - write a systems test for your mapper + reducer together here

In [ ]:
# part e - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB model

In [ ]:
# part e - write your hadoop streaming job

In [ ]:
# part e - extract your results (i.e. model) to a local file

In [ ]:
# part e - print your model (above local file) so that we can confirm that it matches expected results

# Question 9: Enron Ham/Spam NB Classifier & Results.

Fantastic work. We're finally ready to perform Spam Classification on the Enron Corpus. In this question you'll run the analysis you've developed, report its performance.

### Q9 Tasks:
* __a) train/test split:__ Run the provided code to split our Enron file into a training set and testing set then load them into HDFS. [`NOTE:` _Make sure you re calculate the vocab size for just the training set!_]

* __b) train 2 models:__ Write Hadoop Streaming jobs to train MNB Models on the training set with and without smoothing. Save your models to local files at __`NaiveBayes/Unsmoothed/NBmodel.txt`__ and __`NaiveBayes/Smoothed/NBmodel.txt`__. [`NOTE:` _This naming is important because we wrote our classification task so that it expects a file of that name... if this inelegance frustrates you there is an alternative that would involve a few adjustments to your code [read more about it here](http://www.tnoda.com/blog/2013-11-23)._] Finally run the checks that we provide to confirm that your results are correct.


* __c) code:__ Recall that we designed our classification job with just a mapper. An efficient way to report the performance of our models would be to simply add a reducer phase to this job and compute precision and recall right there. Complete the code in __`NaiveBayes/evaluation_reducer.py`__ and then write Hadoop jobs to evaluate your two models on the test set. Report their performance side by side. [`NOTE:` if you need a refresher on precision, recall and F1-score [Wikipedia](https://en.wikipedia.org/wiki/F1_score) is a good resource.]


* __d) short response:__ Compare the performance of your two models. What do you notice about the unsmoothed model's predictions? Can you guess why this is happening? Which evaluation measure do you think is most relevant in our use case? [`NOTE:` _Feel free to answer using your common sense but if you want more information on evaluating the classification task checkout_ [this blogpost](https://tryolabs.com/blog/2013/03/25/why-accuracy-alone-bad-measure-classification-tasks-and-what-we-can-do-about-it/
) or [this paper](http://www.flinders.edu.au/science_engineering/fms/School-CSEM/publications/tech_reps-research_artfcts/TRRA_2007.pdf
)]



### Q9 Student Answers:
> __d)__ Type your answer here!



__Test/Train split__

In [ ]:
# part a - test/train split (RUN THIS CELL AS IS)
!head -n 80 data/enronemail_1h.txt > data/enron_train.txt
!tail -n 20 data/enronemail_1h.txt > data/enron_test.txt
!hdfs dfs -copyFromLocal data/enron_train.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal data/enron_test.txt {HDFS_DIR}

__Training__ (Enron MNB Model _without smoothing_ )

In [ ]:
# part b -  Unsmoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model

# hadoop command
!hadoop jar {JAR_FILE} \








# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-model/part-000* > NaiveBayes/Unsmoothed/NBmodel.txt

In [ ]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000172547666293,0.000296823983378

In [ ]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,8.62738331464e-05,0.00163253190858

__Training__ (Enron MNB Model _with Laplace +1 smoothing_ )

In [ ]:
# part b -  Smoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model

# hadoop command
!hadoop jar {JAR_FILE} \







# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/NBmodel.txt

In [ ]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000185804533631,0.000277300205202

In [ ]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,0.000123869689087,0.00127558094393

__Evaluation__

In [ ]:
# part c - write your code in NaiveBayes/evaluation_reducer.py then RUN THIS
!chmod a+x NaiveBayes/evaluation_reducer.py

In [ ]:
!cat NaiveBayes/evaluation_reducer.py

In [ ]:
# part c - unit test your evaluation job on the chinese model (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py 
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | NaiveBayes/evaluation_reducer.py

In [ ]:
# part c - Evaluate the UNSMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory

# hadoop job





# retrieve results locally

In [ ]:
# part c - Evaluate the SMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory

# hadoop job





# retrieve results locally

In [ ]:
# part c - display results 
# NOTE: feel free to modify the tail commands to match the format of your results file
print('=========== UNSMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Unsmoothed/results.txt
print('=========== SMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Smoothed/results.txt

__`EXPECTED RESULTS:`__ 
<table>
<th>Unsmoothed Model</th>
<th>Smoothed Model</th>
<tr>
<td><pre>
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5
Precision	1.0
Recall	0.0909
F-Score	0.1666
</pre></td>
<td><pre>
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.85
Precision	0.7857
Recall	1.0
F-Score	0.88
</pre></td>
</tr>
</table>

__`NOTE:`__ _Don't be too disappointed if these seem low to you. We've trained and tested on a very very small corpus... bigger datasets coming soon!_

# Question 10: Custom Partitioning and Secondary Sort

Now that we have our model, we can analyse the results and think about future improvements.

### Q10 Tasks:

* __a) code + short response:__ Let's look at the top ten words with the highest conditional probability in `Spam` and in `Ham`. We'll do this by writing a Hadoop job that sorts the model file (`NaiveBayes/Smoothed/NBmodel.py`). Normally we'd have to run two jobs -- one that sorts on $P(word|ham)$ and another that sorts on $P(word|spam)$. However if we slighly modify the data format in the model file then we can get the top words in each class with just one job. We've written a mapper that will do just this for you. Read through __`NaiveBayes/model_sort_mapper.py`__ and then briefly explain how this mapper will allow us to partition and sort our model file. Write a Hadoop job that uses our mapper and `/bin/cat` for a reducer to partition and sort. Print out the top 10 words in each class (where 'top' == highest conditional probability).[`HINT:` _this should remind you a lot of what we did in Question 6._]


* __b) short response:__ What do you notice about the 'top words' we printed in `a`? How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'a'. How would they affect the probabilities of words that occur much more in one class than another? In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model. [`NOTE:` _you do not need to code anything for this task, but if you are struggling with it you could try changing 'k' and see what happens to the test set. We don't recommend doing this exploration with the Enron data because it will be harder to see the impact with such a big vocabulary_]

### Q10 Student Answers:
> __a)__ Type your answer here!

> __b)__ Type your answer here!

In [ ]:
# part a - write your Hadoop job here (sort smoothed model on P(word|class))

# clear output directory

# hadoop job

In [ ]:
# part b - print top words in each class


### Congratulations, you have completed HW2! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform